In [2]:
# <수행 방법 요약>
# 이미지에서 각 모델들이 인식한 객체들에게서 바운딩된 박스, 신뢰도, 라벨들을 추출한다.
# 모든 박스를 서로 한 쌍씩 겹침 정도, 즉 iou를 계산한다.
# iou를 1에서 뺀 값이 특정 수치보다 작으면 많이 겹쳤다고 판단한다.
# 서로 많이 겹쳐진 박스들끼리 그룹화를 한다.
# 각 그룹에서 박스들의 좌표의 평균을 최종 박스 좌표로 결정한다.
# 각 그룹에서 박스들의 신뢰도의 평균을 최종 신뢰도로 결정한다.
# 각 그룹에서 박스들의 라벨 이름중 빈도수가 높은것을 최종 라벨 이름으로 결정한다.(다수결)
# 이미지에 최종 박스 좌표 위치에 사각형을 그리고, 신뢰도와 라벨 이름을 텍스트로 표시한다.

from ultralytics import YOLO
import numpy as np
from scipy.spatial.distance import cdist
from collections import Counter, defaultdict
import cv2

# 모델 파일명 리스트(밴: model10)
model_files = [f'models/model{model_number}.pt' for model_number in [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12]]

# 모델 로드 및 names 저장
models = []
model_names = {}

# 모델파일 이름, 모델, 클래스 저장
for model_file in model_files:
    model = YOLO(model_file)
    models.append((model_file, model)) # 모델 파일 이름도 같이 저장함
    model_names[model_file] = model.names

In [3]:
# 모든 모델 예측 수행, 결과 담기
def ensemble_predict(image):
    results = []
    detection_counts = []
    for model in models:
        print(f'\n{model[0]}', end="")
        result = model[1].predict(image, conf=0.5)
        results.append(result)
        # 각 모델에서 감지한 바운딩 박스의 수
        detection_counts.append(len(result[0].boxes))
    print('\nNumber of objects detected by each model:')
    print(detection_counts)

    # 과반수 이상의 모델이 바운딩 박스가 없는 경우
    if sum(count == 0 for count in detection_counts) >= len(models) / 2: # count가 0인 모델의 수가 과반수 이상인지 검사
        return [], [], []  # 감지된 객체가 없는 경우, 빈 결과 반환

    combined_results = combine_results(*results)  # final_boxes, final_confidences, final_labels
    return combined_results

# 각 결과에서 바운딩 박스, 신뢰도, 라벨 추출
def combine_results(*results):
    combined_boxes = []
    combined_confidences = []
    combined_labels = defaultdict(list)  # 같은 박스 위치에 여러 레이블을 저장하기 위한 딕셔너리

    # 각 모델의 결과에서 바운딩 박스를 추출하여 결합
    for model_index, result_list in enumerate(results):
        model_name = models[model_index][0]
        for result in result_list:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())

                # 클래스 번호를 클래스 이름으로 변환
                class_name = model_names[model_name][class_id]

                combined_boxes.append([x1, y1, x2, y2])
                combined_confidences.append(conf)
                combined_labels[(x1, y1, x2, y2)].append(class_name)

    # if len(combined_boxes) == 0:  # combined_boxes가 비어 있는 경우를 처리합니다.
    #     return [], [], []

    combined_boxes = np.array(combined_boxes)
    combined_confidences = np.array(combined_confidences)

    # 박스를 그룹화하여 다수결로 라벨 결정
    final_boxes = []
    final_confidences = []
    final_labels = []

    box_groups = group_boxes_by_overlap(combined_boxes)  # 겹치는 박스 번호들끼리 그룹화한 리스트

    if len(box_groups) > 0:
        for group in box_groups:
            group_boxes = combined_boxes[group]  # 그룹의 각 번호에 해당하는 박스들의 좌표배열
            group_confidences = combined_confidences[group]  # 그룹의 각 번호에 해당하는 박스들의 신뢰도
            group_labels = [combined_labels[tuple(box)] for box in group_boxes]  # combined_labels = {(박스 좌표배열) : 라벨이름}

            # 그룹 내에서 평균 박스와 평균 신뢰도를 계산
            avg_box = np.mean(group_boxes, axis=0)
            avg_conf = np.mean(group_confidences)

            # 각 박스에 대해 다수결로 레이블 결정
            flattened_labels = [label for sublist in group_labels for label in sublist]  # 라벨이름들이 저장된 리스트
            flattened_labels = [label.lower() for label in flattened_labels] # 라벨이름 모두 소문자로 통일
            print(flattened_labels)
            most_common_label_and_count = Counter(flattened_labels).most_common(1) # 가장 빈도수가 높은 라벨 추출
            most_common_label = most_common_label_and_count[0][0]
            label_count = most_common_label_and_count[0][1]
            if label_count >= 2: # 라벨의 빈도수가 2 이상이면 통과
                final_boxes.append(avg_box)
                final_confidences.append(avg_conf)
                final_labels.append(most_common_label)
            else:
                return [], [], []

        final_boxes = np.array(final_boxes)
        final_confidences = np.array(final_confidences)
        final_labels = np.array(final_labels)
    else:
        return [], [], []

    return final_boxes, final_confidences, final_labels

# 박스가 겹치는 그룹을 찾는 함수(각 그룹 안에는 박스 번호들이 있음)
def group_boxes_by_overlap(boxes, iou_threshold=0.4):
    distances = cdist(boxes, boxes, lambda x, y: 1 - iou(x, y))  # 두 박스간의 겹침 정도(비율)가 클수록 1에서 뺀 값이 작아지므로 거리가 작아진다.
    groups = []
    visited = set()

    for i in range(len(boxes)):  # i: 현재 박스의 인덱스
        if i in visited:  # 이미 방문된 박스 집합(visited)에 있다면 다음 반복으로 넘어감
            continue
        group = [i]  # 현재 박스 번호를 포함한 그룹 생성
        visited.add(i)  # 현재 박스 번호를 집합에 추가
        for j in range(i + 1, len(boxes)):  # 현재 박스의 다음 박스 번호부터 순회
            if j in visited:  # 이미 방문된 박스 집합(visited)에 있다면 다음 반복으로 넘어감
                continue
            if distances[i, j] < iou_threshold:  # 박스 i와 j의 거리가 iou_threshold보다 작다면, 두 박스가 많이 겹친다고 판단함
                group.append(j)  # 박스 번호 j를 group에 추가
                visited.add(j)  # 집합에도 추가
        groups.append(group)  # 그룹을 groups에 추가
    print('\nbox number groups:')
    print(groups)
    print('\nlabels of passed groups:')

    groups = [group for group in groups if len(group) >= 3] # 그룹내 박스가 3개 이상인 것만 통과
    return groups

# IoU (Intersection over Union) 계산 함수
# 객체 탐지 및 이미지 분석에서 두 바운딩 박스 간의 겹침 정도를 측정하는 지표,
# IoU는 두 박스의 교차 영역과 두 박스의 합집합 영역 간의 비율을 나타냄
def iou(box1, box2):
    x1 = max(box1[0], box2[0])  # x1 vs X1, 더 큰 값이 교집합 영역 좌상단 x좌표
    y1 = max(box1[1], box2[1])  # y1 vs Y1, 더 큰 값이 교집합 영역 좌상단 y좌표
    x2 = min(box1[2], box2[2])  # x2 vs X2, 더 작은 값이 교집합 영역 우하단 x좌표
    y2 = min(box1[3], box2[3])  # y2 vs Y2, 더 작은 값이 교집합 영역 우하단 y좌표

    intersection = max(0, x2 - x1) * max(0, y2 - y1)  # 교집합 영역 가로길이 * 교집합 영역 세로길이 = 교집합 영역의 전체 면적
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])  # 박스1의 가로길이 * 세로길이 = 박스1의 전체 면적
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])  # 박스2의 가로길이 * 세로길이 = 박스2의 전체 면적
    union = box1_area + box2_area - intersection  # 두 박스의 합집합 영역에서 교집합 영역을 뺀 값

    return intersection / union if union > 0 else 0  # 교집합 면적을 합집합 면적으로 나눈값

In [20]:
# 이미지 파일을 읽어오기
image_path = 'potato.jpg'
image = cv2.imread(image_path)

# 이미지에 대해 앙상블 예측 수행
combined_results = ensemble_predict(image) # final_boxes, final_confidences, final_labels

# 결과 출력 및 이미지에 그리기
boxes, confidences, labels = combined_results

print("\nfinal conclusion:")
if len(boxes) > 0 and len(confidences) > 0 and len(labels) > 0:
    for box, conf, label in zip(boxes, confidences, labels):
        x1, y1, x2, y2 = map(int, box)
        label_name = label  # 이미 label은 클래스 이름입니다.

        # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
        if label_name is not None:
            # 사각형 테두리 그리는 함수(사각형을 그릴 이미지, 사각형 좌상단 좌표, 사각형 우하단 좌표, 사각형 색, 사각형 두께)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # 텍스트 추가하는 함수(텍스트 추가할 이미지, 텍스트 문자열, 텍스트 시작점, 텍스트 폰트, 텍스트 크기, 텍스트 색상, 텍스트 두께)
            cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # 박스 좌표, 신뢰도, 라벨이름 출력
            print(f'Box: {box}, Confidence: {conf}, Label: {label_name}')

# 수정된 이미지 저장
output_image_path = 'potato_with_boxes_majority.jpg'
cv2.imwrite(output_image_path, image)
print(f"\nAnnotated image saved as {output_image_path}")


models/model1.pt
0: 640x640 (no detections), 142.0ms
Speed: 5.0ms preprocess, 142.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

models/model2.pt
0: 640x640 2 potatos, 89.6ms
Speed: 4.5ms preprocess, 89.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

models/model3.pt
0: 640x640 2 potatos, 82.6ms
Speed: 5.0ms preprocess, 82.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

models/model4.pt
0: 640x640 2 potatos, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

models/model5.pt
0: 640x640 2 Eggs, 19.5ms
Speed: 1.0ms preprocess, 19.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

models/model6.pt
0: 640x640 (no detections), 18.5ms
Speed: 1.0ms preprocess, 18.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

models/model7.pt
0: 640x640 2 potatos, 19.5ms
Speed: 1.5ms preprocess, 19.5ms inference, 1.0ms postprocess per image at shap

In [5]:
# 여러 이미지 예측하기

import os

def process_and_predict_images(image_dir, output_dir):
    # 이미지 디렉토리 내 모든 파일 읽기
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        if os.path.isfile(image_path):
            image = cv2.imread(image_path)

            if image is not None:
                # 이미지에 대해 앙상블 예측 수행
                combined_results = ensemble_predict(image)  # final_boxes, final_confidences, final_labels

                # 결과 출력 및 이미지에 그리기
                boxes, confidences, labels = combined_results

                print(f"\nfinal conclusion for {filename}:")
                if len(boxes) > 0 and len(confidences) > 0 and len(labels) > 0:
                    for box, conf, label in zip(boxes, confidences, labels):
                        x1, y1, x2, y2 = map(int, box)
                        label_name = label  # 이미 label은 클래스 이름입니다.

                        # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
                        if label_name is not None:
                            # 사각형 테두리 그리는 함수
                            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                            # 텍스트 추가하는 함수
                            cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                            # 박스 좌표, 신뢰도, 라벨이름 출력
                            print(f'Box: {box}, Confidence: {conf}, Label: {label_name}')

                # 수정된 이미지 저장
                output_image_path = os.path.join(output_dir, f"predict_{filename}")
                cv2.imwrite(output_image_path, image)
                print(f"Annotated image saved as {output_image_path}")
                print('\n' + '==' * 50)

# 이미지 디렉토리와 출력 디렉토리 지정
image_dir = 'google_test_images'
output_dir = 'google_test_images/predict_majority'

# 출력 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 이미지 처리 및 예측 수행
process_and_predict_images(image_dir, output_dir)


models/model1.pt
0: 384x640 1 carrot, 94.1ms
Speed: 2.0ms preprocess, 94.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

models/model2.pt
0: 384x640 1 Mushroom, 103.0ms
Speed: 3.0ms preprocess, 103.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

models/model3.pt
0: 384x640 (no detections), 66.0ms
Speed: 3.0ms preprocess, 66.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

models/model4.pt
0: 384x640 (no detections), 73.1ms
Speed: 3.0ms preprocess, 73.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

models/model5.pt
0: 384x640 (no detections), 18.5ms
Speed: 1.5ms preprocess, 18.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

models/model6.pt
0: 384x640 (no detections), 23.5ms
Speed: 1.0ms preprocess, 23.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

models/model7.pt
0: 384x640 1 mushroom, 18.5ms
Speed: 1.0ms preprocess, 18.5ms inference, 1.0ms postprocess p